In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# 🧠 Entendimento do Negócio

* Problema: identificar fraudes com base em scores e comportamento de compra/documentos.

* Variável-alvo: fraude

In [ ]:
import pandas as pd

csv_url = (
    'https://docs.google.com/spreadsheets/d/'
    '1HqSaTwX5edc-zpO8BQHvMIbndjZFLslxoHLcuUwO5pY'
    '/export?format=csv&gid=2136114581'
)
df = pd.read_csv(csv_url)
df.head()


# 🔍 Análise Exploratória (EDA)

In [ ]:

# 1) Estatísticas descritivas básicas
print(df.describe())

* Verificação de valores ausentes (missing values)

In [ ]:
import numpy as np

# Função para verificar valores ausentes
def check_missing_values(df):
    return df.isnull().sum()

# Verificando valores ausentes
missing_values = check_missing_values(df)
print("Valores ausentes:\n", missing_values)

In [ ]:
# Imputação de valores ausentes
from sklearn.impute import SimpleImputer

# Para colunas numéricas
numerical_cols_with_missing = ['score_2', 'score_3', 'score_4', 'score_6', 'score_9', 'score_10']
imputer_numeric = SimpleImputer(strategy='median')  # Usando a mediana como estratégia de imputação

df[numerical_cols_with_missing] = imputer_numeric.fit_transform(df[numerical_cols_with_missing])

# Para colunas categóricas
categorical_cols_with_missing = ['pais', 'entrega_doc_2']
imputer_categorical = SimpleImputer(strategy='most_frequent')  # Usando a moda (valor mais frequente)

df[categorical_cols_with_missing] = imputer_categorical.fit_transform(df[categorical_cols_with_missing])

# Verificando os valores ausentes após imputação
missing_values_after = df.isnull().sum()
print("\nValores ausentes após imputação:\n", missing_values_after)


* Imputação numérica: Utilizando a mediana para preencher os valores ausentes, pois é mais robusta contra outliers.

* Imputação categórica: Usando o valor mais frequente (modo) para preencher os valores ausentes em colunas categóricas.

* Analise Outliers



In [ ]:
from scipy.stats import mstats

# Lista das colunas numéricas a serem analisadas
numerical_cols = ['score_2', 'score_3', 'score_4', 'score_6', 'score_9', 'score_10']

# Aplicando winsorização nas colunas numéricas
for col in numerical_cols:
    df[col] = mstats.winsorize(df[col], limits=[0.05, 0.05])  # Winsoriza 5% dos valores em cada extremidade

# Verificando se os valores extremos foram ajustados
print("\nEstatísticas após Winsorização:")
print(df[numerical_cols].describe())


* Winsorização: Os valores extremos (5% dos valores mais baixos e mais altos) são ajustados para os limites dentro desse intervalo. Isso ajuda a evitar que outliers distorçam o modelo.

* Resultados após Winsorização:
* As variáveis como score_2, score_3, score_6, score_9 e score_10 apresentaram mudanças significativas nos valores máximos e na dispersão, alinhando-se melhor aos intervalos definidos.

* O valor máximo de score_6 foi ajustado de 145274 para 162, o que é uma evidência de que os outliers mais extremos foram tratadas.

* A distribuição de outras variáveis, como score_2, também mostra uma compressão dos valores, o que indica que os outliers foram reduzidos.

#_______________________________________________________#

# 📊 Análise Univariada

In [ ]:
# Estatísticas descritivas
variaveis_univariadas = [
    'score_3', 'valor_compra', 'score_fraude_modelo', 'score_10', 'score_6', 'score_9', 'entrega_doc_1'
]

# Exibe as estatísticas descritivas para as variáveis selecionadas
print(df[variaveis_univariadas].describe())




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráficos: Histograma + KDE
for col in variaveis_univariadas:
    plt.figure(figsize=(16, 6))

    # Histograma e KDE
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, color='skyblue')
    plt.title(f'Distribuição de {col}')

    plt.tight_layout()
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráficos: Histograma + KDE
for col in variaveis_univariadas:
    plt.figure(figsize=(16, 6))

    # Histograma e KDE
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, color='skyblue')
    plt.title(f'Distribuição de {col}')

    plt.tight_layout()
    plt.show()



* Resultados - estatísticas descritivas (média, desvio padrão, mínimo, 25%, 50%, 75%, máximo) das variáveis-chave.

* Histograma e KDE: O qual mostra a distribuição das variáveis, tanto com um histograma quanto com a estimativa de densidade (KDE), para entender a forma e a dispersão da variável.

* Boxplot: Exibe a dispersão e possíveis outliers nas variáveis, facilitando a visualização de valores extremos.


 # 🔗 Análise Bivariada




In [ ]:
#  Análise Bivariada (Correlação com a variável alvo fraude)
# Seleciona apenas colunas numéricas
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Calcula correlação com a variável fraude
correlations = df[numeric_cols].corr()['fraude'].sort_values(ascending=False)

# Exibe as correlações (positivas e negativas mais fortes)
print(correlations)


# O modelo score_fraude_modelo está alinhado com a variável alvo, como esperado.

Há indícios de que:

Compras altas têm mais chance de fraude.

Não entrega de documentos está fortemente associada à fraude.

Alguns scores operacionais (score_6, score_10, score_9) têm correlação negativa interessante com a fraude — podem indicar comportamento legítimo.

* Médias dos scores por classe de fraude

In [ ]:
# Média das variáveis numéricas por grupo de fraude
means_by_class = df.groupby('fraude')[numeric_cols].mean().T
means_by_class.columns = ['Não Fraude', 'Fraude']
means_by_class['Diferença'] = means_by_class['Fraude'] - means_by_class['Não Fraude']

print(means_by_class.sort_values(by='Diferença', ascending=False))


# Estratégia

* A não entrega do documento (entrega_doc_1) é um dos melhores indicadores de fraude até agora.

* Scores como score_6, score_10, score_9 têm grande poder discriminativo — podem ser valiosos em modelos.

* A diferença de valor_compra também é substancial — talvez sirva como feature derivada (e.g. log(valor), categorias de valor).

* Modelo atual (score_fraude_modelo) tem desempenho razoável, mas talvez seja possível superá-lo.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Lista das variáveis mais relevantes
variaveis_chave = ['score_3', 'valor_compra', 'score_fraude_modelo',
                   'entrega_doc_1', 'score_6', 'score_10', 'score_9']

# Estilo
sns.set(style="whitegrid")

# Criar boxplots
for var in variaveis_chave:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='fraude', y=var, data=df, palette="Set2")
    plt.title(f'Distribuição de {var} por Fraude')
    plt.xlabel('Fraude (0 = Não, 1 = Sim)')
    plt.ylabel(var)
    plt.tight_layout()
    plt.show()


* Teve uma diferenças significativa entre fraude e não fraude: A maioria das variáveis mostra diferenças significativas entre as classes, o que sugere que elas podem ser potencialmente úteis para prever fraudes.

* As distribuições não normais e variâncias não homogêneas: Como as distribuições não seguem uma normalidade e as variâncias não são homogêneas, podemos usar alguns modelos não paramétricos (como árvores de decisão, Random Forests ou XGBoost) ou técnicas que não dependem de pressupostos de normalidade.

* Importância de variáveis como score_fraude_modelo e valor_compra: Algumas variáveis, como score_fraude_modelo e valor_compra, apresentam grandes diferenças entre fraude e não fraude, o que as torna importantes para modelagem preditiva.

* Breve resumo:
Significância: Todas as variáveis mostraram diferenças significativas entre fraude e não fraude.



# 📈 Testes de Hipóteses

In [ ]:
from scipy.stats import mannwhitneyu

# Lista de variáveis para os testes
variaveis_para_teste = [
    'score_3', 'valor_compra', 'score_fraude_modelo', 'score_10', 'score_6', 'score_9', 'entrega_doc_1'
]

# Resultados dos testes de hipóteses
testes_hipotese = []

# Executando o teste de Mann-Whitney para cada variável
for var in variaveis_para_teste:
    # Separando as classes
    fraude = df[df['fraude'] == 1][var]
    nao_fraude = df[df['fraude'] == 0][var]

    # Realizando o teste de Mann-Whitney
    stat, p_valor = mannwhitneyu(fraude, nao_fraude)

    # Armazenando os resultados
    testes_hipotese.append({
        'variavel': var,
        'p_normal_0': nao_fraude.kurtosis(),  # Teste de normalidade para classe 0
        'p_normal_1': fraude.kurtosis(),  # Teste de normalidade para classe 1
        'p_levene': stat,  # estatística de Mann-Whitney
        'teste': 'Mann-Whitney',
        'p_valor_teste': p_valor
    })

# Exibindo os resultados
resultados_hipotese = pd.DataFrame(testes_hipotese)
print(resultados_hipotese)


* Importante Observações:

- Todos os p-valores estão abaixo de 0.05 ⇒ rejeitamos H₀, ou seja, há diferenças estatisticamente significativas entre os grupos.

- As kurtoses reforçam que os dados têm distribuições diferentes (especialmente valor_compra e score_6), justificando o uso do teste não paramétrico.

# ✅ O que isso diz para o projeto:

- As variáveis testadas são bons candidatos para features de modelagem, pois mostram comportamentos estatisticamente diferentes entre classes.



* Teste de hipótese
1. Comparou as distribuições das variáveis entre as classes
Para cada variável, o teste avaliou:

“A distribuição dessa variável é significativamente diferente entre os grupos com e sem fraude?”

2. Evidênciou estatística para apoiar as features

* O resultado mais importante é o p-valor:

- Sempre se o p_valor_teste < 0.05, significa que há evidência estatística de que a variável tem comportamento diferente entre as classes.

- Isso indica que a variável pode ser útil para detectar fraude.



* O teste de hipótese mostrou que todas as variáveis analisadas têm diferenças estatisticamente significativas entre as classes fraude e não fraude (p-valores muito baixos). Isso indica que essas variáveis são boas candidatas para alimentar o modelo, pois ajudam a distinguir entre comportamentos fraudulentos

#___________________________________#

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd

# 1. Selecionar variáveis e target
X = df[['score_3', 'valor_compra', 'score_fraude_modelo', 'score_10', 'score_6', 'score_9', 'entrega_doc_1']]
y = df['fraude']

# 2. Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# 3. Modelos
modelos = {
    'Regressão Logística': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# 4. Treinar, prever e comparar
resultados = []

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_proba = modelo.predict_proba(X_test)[:,1]

    metrics = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, y_proba)

    resultados.append({
        'Modelo': nome,
        'Acurácia': metrics['accuracy'],
        'Precisão (fraude)': metrics['1']['precision'],
        'Recall (fraude)': metrics['1']['recall'],
        'F1-Score (fraude)': metrics['1']['f1-score'],
        'AUC-ROC': auc
    })

# Exibir resultados
df_resultados = pd.DataFrame(resultados)
print(df_resultados.sort_values(by='AUC-ROC', ascending=False))


* Resultados dos modelos

- ✅ Melhor Modelo: Random Forest
Acurácia: 95,17%

F1-Score para fraude: 0.19 (melhor equilíbrio entre precisão e recall)

AUC-ROC: 0.81 (classificação geral boa)

- 🔍 Resultados mostram:
Todos os modelos acertam bem o “não fraude” (explica a alta acurácia)

- Random Forest teve o melhor desempenho geral para detectar fraude, com melhor recall, precisão e F1.

- Logística nao teve um bom recall (0.005) – ou seja, quase não identificou fraudes.

- XGBoost foi muito bom também, só levemente atrás do Random Forest.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Treinando os modelos
modelo_log = LogisticRegression(random_state=42)
modelo_log.fit(X_train, y_train)

modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train, y_train)

modelo_xgb = XGBClassifier(random_state=42)
modelo_xgb.fit(X_train, y_train)

# Lista de modelos e seus nomes
modelos = [
    ('Regressão Logística', modelo_log),
    ('Random Forest', modelo_rf),
    ('XGBoost', modelo_xgb)
]

# Plot das matrizes de confusão
plt.figure(figsize=(18, 5))

for i, (nome, modelo) in enumerate(modelos):
    # Previsões
    y_pred = modelo.predict(X_test)

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)

    # Plot
    plt.subplot(1, 3, i+1)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=modelo.classes_)
    disp.plot(ax=plt.gca(), values_format='d')
    plt.title(nome)

plt.tight_layout()
plt.show()



* Esses sao os resultados de falso negativo (fraude náo detectada) Como apresentado o Xgboost teve menor numero de fraudes

# Teste de hiperparamentros

In [ ]:
# Balanceamento da base com SMOTE
from imblearn.over_sampling import SMOTE

# Aplicando o SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

# Verificando o balanceamento
print("Distribuição após SMOTE:")
print(y_train_bal.value_counts())


In [ ]:
# Re-treinar Random Forest e XGBoost com dados balanceados

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Reinstanciando e treinando os modelos
modelo_rf_bal = RandomForestClassifier(random_state=42)
modelo_rf_bal.fit(X_train_bal, y_train_bal)

modelo_xgb_bal = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
modelo_xgb_bal.fit(X_train_bal, y_train_bal)


In [ ]:
# Avaliar desempenho dos modelos balanceados
from sklearn.metrics import classification_report, roc_auc_score

modelos_balanceados = {
    'Random Forest (SMOTE)': modelo_rf_bal,
    'XGBoost (SMOTE)': modelo_xgb_bal
}

for nome, modelo in modelos_balanceados.items():
    y_pred = modelo.predict(X_test)
    print(f"\nModelo: {nome}")
    print(classification_report(y_test, y_pred, digits=4))

    # AUC
    if hasattr(modelo, "predict_proba"):
        y_prob = modelo.predict_proba(X_test)[:, 1]
    else:
        y_prob = modelo.decision_function(X_test)
    auc = roc_auc_score(y_test, y_prob)
    print("AUC-ROC:", round(auc, 4))


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Separando treino e teste antes de aplicar SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Aplicando SMOTE no conjunto de treino
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Treinando modelos com dados balanceados (SMOTE)
modelo_rf_smote = RandomForestClassifier(random_state=42)
modelo_rf_smote.fit(X_train_res, y_train_res)

modelo_xgb_smote = XGBClassifier(random_state=42)
modelo_xgb_smote.fit(X_train_res, y_train_res)



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Random Forest com SMOTE
modelo_rf_smote = RandomForestClassifier(random_state=42)
modelo_rf_smote.fit(X_train_res, y_train_res)

# XGBoost com SMOTE
modelo_xgb_smote = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
modelo_xgb_smote.fit(X_train_res, y_train_res)


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Lista de modelos treinados com SMOTE
modelos_smote = [
    ('Random Forest (SMOTE)', modelo_rf_smote),
    ('XGBoost (SMOTE)', modelo_xgb_smote)
]

# Avaliação dos modelos
for nome, modelo in modelos_smote:
    print(f"\nModelo: {nome}")

    # Previsões
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1]  # Probabilidades para AUC-ROC

    # Relatório de classificação
    print(classification_report(y_test, y_pred, digits=4))

    # AUC-ROC
    auc = roc_auc_score(y_test, y_prob)
    print(f"AUC-ROC: {auc:.4f}")

    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=modelo.classes_)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(nome)
    plt.show()


 * XGBoost teve ligeiramente melhor recall de fraudes, mas menor precisão.

Random Forest teve melhor equilíbrio geral (AUC, F1 e acurácia).

A diferença de acertos totais é pequena.

* 💡 Se o foco é detectar mais fraudes mesmo errando um pouco mais (alto recall), o XGBoost pode ser preferido.
Se o foco é ter uma performance mais equilibrada, o Random Forest se destaca.


* Análise do Trade-off:
* Precision vs. Recall: Embora a precision para a classe "não fraude" seja muito boa, a recall da classe "fraude" precisa ser muito melhorada. Ou seja, o modelo é conservador (não identifica muitas fraudes) e perde a chance de bloquear muitas transações fraudulentas.

* Custo das fraudes: Falsos negativos (fraudes não identificadas) são mais críticos do que falsos positivos (compra legítima erradamente marcada como fraude). O prejuízo por fraude aprovada é 100% do valor da compra, então, precisamos otimizar os modelos para identificar mais fraudes, mesmo que isso signifique reduzir a precisão.

# Ajuste do cutoff de decisão: Uma abordagem para melhorar o recall de fraudes seria ajustar o cutoff de probabilidade.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Montar o df_test com tudo que precisamos
df_test = X_test.copy()
df_test['fraude'] = y_test.values
# Traga o valor_compra do df original pelo mesmo índice
df_test['valor_compra'] = df.loc[X_test.index, 'valor_compra']
# Probabilidade de fraude segundo o modelo balanceado
df_test['prob_rf'] = modelo_rf_smote.predict_proba(X_test)[:, 1]

# 2) Função de lucro líquido
def lucro_liquido(cutoff):
    aprovadas = df_test[df_test['prob_rf'] < cutoff]
    ganho = aprovadas.loc[aprovadas['fraude'] == 0, 'valor_compra'].sum() * 0.10
    perda = aprovadas.loc[aprovadas['fraude'] == 1, 'valor_compra'].sum()
    return ganho - perda

# 3) Simular lucro para uma grade de cutoffs de 0 a 1
thresholds = np.linspace(0, 1, 101)
profits = [lucro_liquido(t) for t in thresholds]

# 4) Encontrar o melhor cutoff
best_i = int(np.argmax(profits))
best_cutoff = thresholds[best_i]
best_profit = profits[best_i]

print(f"🔎 Melhor cutoff: {best_cutoff:.2f}")
print(f"💰 Lucro líquido estimado: R$ {best_profit:,.2f}")

# 5) Plotar Lucro vs Cutoff
plt.figure(figsize=(8,5))
plt.plot(thresholds, profits, marker='o')
plt.axvline(best_cutoff, color='red', linestyle='--', label=f"Cutoff ótimo ({best_cutoff:.2f})")
plt.title('Lucro Líquido vs. Cutoff de Probabilidade de Fraude')
plt.xlabel('Cutoff de Probabilidade')
plt.ylabel('Lucro Líquido (R$)')
plt.legend()
plt.grid(True)
plt.show()


* Regra de decisão: Ao bloquear todas as transações em que a probabilidade de fraude ≥ 0.44, você maximiza o retorno financeiro.
* Trade‑off: Esse ponto reflete um equilíbrio em que você aprende a aceitar algumas fraudes (para não perder compras legítimas demais), mas bloqueia fraudes suficientes para gerar o maior lucro líquido.

In [ ]:
#classificando as transações com base na probabilidade de fraude,
y_proba = modelo_rf_smote.predict_proba(X)[:,1]
aprovar = (y_proba < 0.44)


# 🧠 Insights Finais

- Variáveis mais discriminativas

- Entrega de documentos (entrega_doc_1): ausência de documento aumenta fortemente a probabilidade de fraude.

- Scores operacionais (score_6, score_9, score_10, score_3): perfis de comportamento (“notas” de confiabilidade) mostram diferenças significativas entre fraudadores e clientes legítimos.

- Valor da compra: transações fraudulentas tendem a ter ticket médio muito maior.

- Modelagem e performance

- Random Forest (SMOTE): melhor equilíbrio geral (AUC‑ROC 0.7475, Recall fraude 35.9%).

- XGBoost (SMOTE): ligeiramente maior recall (38.7%) mas AUC‑ROC um pouco menor (0.7379).

- Regressão Logística: baixa capacidade de capturar fraudes (recall < 1%).

Otimização para o negócio

Cutoff ótimo de 0.44 sobre a probabilidade de fraude do Random Forest (SMOTE), que maximiza o lucro líquido em R$ 68.525,43 no conjunto de teste.

Isso ajuda a decidir equilibra aprovar compras legítimas (ganho de 10%) e bloquear fraudes (evitar perda de 100%).



#_________________________________________________________________________________________________________________________________________________________#

# 💼 Sugestão ao Negócio

* Implementar o modelo em produção

Score em tempo real via API: para cada nova transação, calcular prob_fraude = modelo.predict_proba(...)[:,1].

Regra de negócio: aprovar se prob_fraude < 0.44; caso contrário, bloquear ou escalonar para análise manual.

* Monitoramento contínuo

Métricas operacionais: acompanhar diariamente recall de fraude, ratio de aprovações bloqueadas, receita gerada.

Data‑drift: monitorar distribuição de features (scores, valor_compra, entrega_doc_) para detectar mudanças no perfil de fraude.

* Ciclo de retroalimentação

Feedback loop: incorporar rapidamente o resultado real (confirmado como fraude ou não) para re-treinar o modelo periodicamente.

- A/B testing de cutoffs: testar limiares ligeiramente diferentes (por exemplo, 0.40, 0.45) em grupos de controle para validar o cutoff ótimo em produção.

* Aprimorar features e cobertura

- Explorar novas variáveis derivadas dos scores (combinações, médias ponderadas).

- Incluir dados comportamentais (tempo de navegação, device fingerprinting) se disponíveis.

# Governança e compliance

Documentar todo o fluxo de dados e modelo para auditoria.

Garantir explicabilidade usando SHAP para justificar bloqueios de transações, atendendo a requisitos regulatórios.



# ✅ Próximos passos recomendados:
* Testar estabilidade do cutoff em diferentes splits ou períodos (time split).

* Monitorar performance no ambiente real, ajustando se houver drift de dados.

* Avaliar impacto de novas features para tentar elevar ainda mais esse lucro.

# 🚀 Deploy

- Infraestrutura: containerizar o modelo (Docker) e expô-lo via REST API (FastAPI ou Flask)

- Latência: garantir inferência em 200 ms por transação, priorizando predição em batch e em tempo real, se necessário

- Escalabilidade: usar orquestradores (Kubernetes) para suportar picos de demanda

- Logging e alertas: capturar decisões, probabilidades, contexto da transação e acionar alertas para aumento súbito de fraudes.